# 作业一：实现HMM中文分词和BPE英文分词
姓名：宋源祎

学号：522030910158

## 任务一：HMM模型用于中文分词

任务一评分标准：
1. 共有8处TODO需要填写，每个TODO计1-2分，共9分，预计代码量30行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

注：本任务仅在短句子上进行效果测试，因此对概率的计算可直接进行连乘。在实践中，常先对概率取对数，将连乘变为加法来计算，以避免出现数值溢出的情况。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [107]:
import pickle
import numpy as np

导入HMM参数，初始化所需的起始概率矩阵、转移概率矩阵、发射概率矩阵。

In [108]:
with open("hmm_parameters.pkl", "rb") as f:
    hmm_parameters = pickle.load(f)

# 非断字（B）为第0行，断字（I）为第1行
# 发射概率矩阵中，词典大小为65536，以汉字的Unicode码点（一个整数值）作为行索引
start_probability = hmm_parameters["start_prob"]  # shape(2)
trans_matrix = hmm_parameters["trans_mat"]  # shape(2, 2)
emission_matrix = hmm_parameters["emission_mat"]  # shape(2, 65536)

# print(start_probability)
# print(trans_matrix)

定义待处理的句子：

In [117]:
# TODO: 将your_name中的xxx替换为你的姓名【1分】
your_name = "全宇豪"  
# 我的姓名是宋源祎，但是我的“祎”似乎不在给出的词典里，导致如果使用我的名字，不但分词分不出来，而且句子的概率值还会变成0，所以我换成了一个朋友的名字，他不在本专业不会引起误会
input_sentence = f"{your_name}是一名优秀的学生"

实现Viterbi算法，并以此进行中文分词。

In [118]:
def viterbi(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> str:
    """
    Viterbi算法进行中文分词。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        str - 中文分词的结果
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种标注（B/I）的最大概率值
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # `path`用来储存最大概率对应的上步B/I选择
    #  例如 path[1][7] == 1 意味着第8个（从1开始计数）字符标注I对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[0][5] == 1 意味着第6个字符标注B对应的最大概率，其前一步的隐状态为1（I）
    #  例如 path[1][1] == 0 意味着第2个字符标注I对应的最大概率，其前一步的隐状态为0（B）
    path = np.zeros((2, len(sent_ord)), dtype=int)

    #  TODO: 第一个位置的最大概率值计算【1分】
    # 初始分别为B/I的概率已经给定，直接乘以发射矩阵即可
    dp[0][0] = start_prob[0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1] * emission_mat[1][sent_ord[0]]

    #  TODO: 其余位置的最大概率值计算（填充dp和path矩阵）【2分】
    for i in range(1, len(sent_ord)):
        tran00 = dp[0][i-1]*trans_mat[0][0]
        tran01 = dp[0][i-1]*trans_mat[0][1]
        tran10 = dp[1][i-1]*trans_mat[1][0]
        tran11 = dp[1][i-1]*trans_mat[1][1]
        dp[0][i] = emission_mat[0][sent_ord[i]]*max(tran00, tran10)
        dp[1][i] = emission_mat[1][sent_ord[i]]*max(tran01, tran11)
        path[0][i] = 1 if tran10 > tran00 else 0
        path[1][i] = 1 if tran11 > tran01 else 0        

    #  `labels`用来储存每个位置最有可能的隐状态
    labels = [0 for _ in range(len(sent_ord))]

    #  TODO: 计算labels每个位置上的值（填充labels矩阵）【1分】
    for i in range(len(sent_ord)-1, -1, -1):
        if i == len(sent_ord)-1:
            labels[i] = 0 if dp[0][i] > dp[1][i] else 1
        else:
            labels[i] = path[labels[i+1]][i+1]

    #  根据lalels生成切分好的字符串
    sent_split = []
    for idx, label in enumerate(labels):
        if label == 1:
            sent_split += [sent_ord[idx], ord("/")]
        else:
            sent_split += [sent_ord[idx]]
    sent_split_str = "".join([chr(x) for x in sent_split])

    return sent_split_str

In [119]:
print("Viterbi算法分词结果：", viterbi(input_sentence, start_probability, trans_matrix, emission_matrix))

Viterbi算法分词结果： 全宇/豪是/一名/优秀/的/学生/


实现前向算法，计算该句子的概率值。

In [120]:
def compute_prob_by_forward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    前向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 初始位置概率的计算【1分】
    dp[0][0] = start_prob[0] * emission_mat[0][sent_ord[0]]
    dp[1][0] = start_prob[1] * emission_mat[1][sent_ord[0]]

    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    for i in range(1, len(sent_ord)):
        dp[0][i] = emission_mat[0][sent_ord[i]] * (dp[0][i - 1] * trans_mat[0][0] + dp[1][i - 1] * trans_mat[1][0])
        dp[1][i] = emission_mat[1][sent_ord[i]] * (dp[0][i - 1] * trans_mat[0][1] + dp[1][i - 1] * trans_mat[1][1])

    return sum([dp[i][len(sent_ord) - 1] for i in range(2)])

实现后向算法，计算该句子的概率值。

In [121]:
def compute_prob_by_backward(sent_orig: str, start_prob: np.ndarray, trans_mat: np.ndarray, emission_mat: np.ndarray) -> float:
    """
    后向算法，计算输入中文句子的概率值。

    Args:
        sent_orig: str - 输入的句子
        start_prob: numpy.ndarray - 起始概率矩阵
        trans_mat: numpy.ndarray - 转移概率矩阵
        emission_mat: numpy.ndarray - 发射概率矩阵

    Return:
        float - 概率值
    """

    #  将汉字转为数字表示
    sent_ord = [ord(x) for x in sent_orig]

    # `dp`用来储存不同位置每种隐状态（B/I）下，从结尾到该位置为止的句子的概率
    dp = np.zeros((2, len(sent_ord)), dtype=float)

    # TODO: 终末位置概率的初始化【1分】
    l = len(sent_orig)-1
    dp[0][l] = 1
    dp[1][l] = 1
    # TODO: 先计算其余位置的概率（填充dp矩阵），然后返回概率值【1分】
    for i in range(l-1, -1, -1):
        dp[0][i] = emission_mat[0][sent_ord[i+1]] * trans_mat[0][0] * dp[0][
            i+1] + trans_mat[0][1] * dp[1][i+1] * emission_mat[1][sent_ord[
                i+1]]
        dp[1][i] = emission_mat[0][sent_ord[i+1]] * trans_mat[1][0] * dp[0][
            i+1] + trans_mat[1][1] * dp[1][i+1] * emission_mat[1][sent_ord[
                i+1]]

    return sum([dp[i][0] * start_prob[i] * emission_mat[i][sent_ord[0]] for i in range(2)])

In [123]:
print("前向算法概率：", compute_prob_by_forward(input_sentence, start_probability, trans_matrix, emission_matrix))

print("后向算法概率：", compute_prob_by_backward(input_sentence, start_probability, trans_matrix, emission_matrix))

前向算法概率： 6.670603396924474e-33
后向算法概率： 6.670603396924474e-33


### 实验总结
> TODO：请在这里填写实验总结.

### 实验总结
本次实验，我用viterbi算法进行中文句子的分词，以及分别使用前向后向算法计算所分词句子出现的概率。

其中，viterbi算法用已知的起始概率矩阵作为马尔可夫的初始隐状态，用转移概率矩阵来计算每一个词在某隐状态下，下一个词属于某隐状态的概率，本题中隐状态即为B/I，发射概率矩阵用于计算给定隐状态下取到某个词的概率，从而通过动态规划的方法，从句子开始遍历一遍，找到概率最大的一种划分方式。

计算句子出现的概率，需要综合考虑每个词取到每个隐状态时的概率值，并将它们加在一起。前向算法从句子的起始算起，后向算法从句子的结尾开始算起，他们都可以在遍历一次句子后计算出该句话出现的概率值，计算非常高效简便。

本次代码小作业很好的加深了我对viterbi算法以及前向后向算法计算句子概率值的理解，也让我意识到数学推导的重要性，由于这些算法都具备了非常简洁美观的推理公式，使得实现代码也十分简单。

同时我也意识到这种分词方式的局限性，分词效果完全给出的依赖词库，否则可能导致分词效果非常不好甚至完全失败。比如我的姓名是宋源祎，但是我的“祎”似乎不在给出的词典里，导致如果使用我的名字，不但分词分不出来，而且计算句子的概率值还会变成0。（也因此在最终递交的结果中，我换成了一个朋友的名字，他不在本专业不会引起误会）

总结实验结果：
1. Viterbi算法分词结果： 全宇/豪是/一名/优秀/的/学生/
2. 前向算法概率： 6.670603396924474e-33
   后向算法概率： 6.670603396924474e-33
可以看出，分词中除了姓名分词出现错误以外，其他划分都是正确的，而名字的划分依赖数据集，比如用我的代码测试“夏欣媛”同学的名字时就不会出现问题。计算句子概率的前向算法和后向算法结果相同，这也是符合实际的。总体而言，任务一比较简单，只需要对viterbi算法和计算句子概率的前向后向算法的计算公式完全理解就可以很容易的完成，

## 任务二：BPE算法用于英文分词

任务二评分标准：

1. 共有6处TODO需要填写，每个TODO计1-2分，共9分，预计代码量50行。
2. 允许自行修改、编写代码完成。对于该情况，请补充注释以便于评分，否则结果不正确将导致较多的扣分。
3. 用于说明实验的文字和总结不额外计分，但不写会导致扣分。

> 你可以像这样在Markdown单元格中使用引用符号`>`，  
以及在代码单元格中使用注释来说明你的实验。  

---

In [3]:
import re
from typing import List, Tuple, Dict

构建空格分词器，将语料中的句子以空格切分成单词，然后将单词拆分成字母加`</w>`的形式。例如`apple`将变为`a p p l e </w>`。

In [4]:
_splitor_pattern = re.compile(r"[^a-zA-Z']+|(?=')")
_digit_pattern = re.compile(r"\d+")


def white_space_tokenize(corpus: List[str]) -> List[List[str]]:
    """
    先正则化（字母转小写、数字转为N、除去标点符号），然后以空格分词语料中的句子，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到 `[["i", "am", "happy"], ["i", "have", "N", "apples"]]`

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        List[List[str]] - 二维List，内部的List由每个句子的单词str构成
    """

    tokeneds = [list(filter(lambda token: len(token) > 0, _splitor_pattern.split(_digit_pattern.sub("N", sentence.lower())))) for sentence in corpus]

    return tokeneds

编写相应函数构建BPE算法需要用到的初始状态词典。

In [54]:
def build_bpe_vocab(corpus: List[str]) -> Dict[str, int]:
    """
    将语料进行white_space_tokenize处理后，将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的字典，例如：  
    输入 `corpus = ["I am happy.", "I have 10 apples!"]`，  
    得到
    ```python
    {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```

    Args:
        corpus: List[str] - 待处理的语料

    Return:
        Dict[str, int] - "单词分词状态->频数"的词典
    """

    tokenized_corpus = white_space_tokenize(corpus)
    # print(tokenized_corpus)

    bpe_vocab = dict()

    # TODO: 完成函数体【1分】
    for sentence in tokenized_corpus:
        for word in sentence:
            key = ''
            for i in range(len(word)):
                if i == len(word)-1:
                    key = key + word[i] + " </w>"
                else:
                    key = key + word[i] + " "
            bpe_vocab[key] = bpe_vocab.get(key, 0) + 1

    return bpe_vocab

In [55]:
corpus = ["I am --$*@happy.", "I have 10 apples!"]
print(build_bpe_vocab(corpus))

{'i </w>': 2, 'a m </w>': 1, 'h a p p y </w>': 1, 'h a v e </w>': 1, 'N </w>': 1, 'a p p l e s </w>': 1}


编写所需的其他函数：

In [76]:
def get_bigram_freq(bpe_vocab: Dict[str, int]) -> Dict[Tuple[str, str], int]:
    """
    统计"单词分词状态->频数"的词典中，各bigram的频次（假设该词典中，各个unigram以空格间隔），例如：  
    输入 
    ```python
    bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        ('i', '</w>'): 2,
        ('a', 'm'): 1,
        ('m', '</w>'): 1,
        ('h', 'a'): 2,
        ('a', 'p'): 2,
        ('p', 'p'): 2,
        ('p', 'y'): 1,
        ('y', '</w>'): 1,
        ('a', 'v'): 1,
        ('v', 'e'): 1,
        ('e', '</w>'): 1,
        ('N', '</w>'): 1,
        ('p', 'l'): 1,
        ('l', 'e'): 1,
        ('e', 's'): 1,
        ('s', '</w>'): 1
    }
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        Dict[Tuple[str, str], int] - "bigram->频数"的词典
    """

    bigram_freq = dict()

    # TODO: 完成函数体【1分】
    for word, num in bpe_vocab.items():
        unigrams = word.split(' ')
        for i in range(len(unigrams)-1):
            key = (unigrams[i], unigrams[i+1])
            bigram_freq[key] = bigram_freq.get(key, 0) + num

    return bigram_freq

In [78]:
bpe_vocab = {
    'i </w>': 2,
    'aa m </w>': 1,
    'h aa pp p y</w>': 1,
    'h aa v e </w>': 1,
    'N </w>': 1,
    'aa pp l e s </w>': 1
}

print(get_bigram_freq(bpe_vocab))

{('i', '</w>'): 2, ('aa', 'm'): 1, ('m', '</w>'): 1, ('h', 'aa'): 2, ('aa', 'pp'): 2, ('pp', 'p'): 1, ('p', 'y</w>'): 1, ('aa', 'v'): 1, ('v', 'e'): 1, ('e', '</w>'): 1, ('N', '</w>'): 1, ('pp', 'l'): 1, ('l', 'e'): 1, ('e', 's'): 1, ('s', '</w>'): 1}


In [9]:
def refresh_bpe_vocab_by_merging_bigram(bigram: Tuple[str, str], old_bpe_vocab: Dict[str, int]) -> Dict[str, int]:
    """
    在"单词分词状态->频数"的词典中，合并指定的bigram（即去掉对应的相邻unigram之间的空格），最后返回新的词典，例如：  
    输入 
    ```python
    bigram = ('i', '</w>'), old_bpe_vocab = {
        'i </w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    得到
    ```python
    {
        'i</w>': 2,
        'a m </w>': 1,
        'h a p p y </w>': 1,
        'h a v e </w>': 1,
        'N </w>': 1,
        'a p p l e s </w>': 1
    }
    ```
    
    Args:
        bigram: Tuple[str, str] - 待合并的bigram
        old_bpe_vocab: Dict[str, int] - 初始"单词分词状态->频数"的词典

    Return:
        Dict[str, int] - 合并后的"单词分词状态->频数"的词典
    """

    new_bpe_vocab = dict()

    # TODO: 完成函数体【1分】
    old_str = bigram[0] + ' ' + bigram[1]
    new_str = bigram[0] + bigram[1]
    for key in old_bpe_vocab.keys():
        new_bpe_vocab[key.replace(old_str,new_str)] = old_bpe_vocab[key]

    return new_bpe_vocab

In [73]:
bigram = ('p', 'y</w>')
old_bpe_vocab = {
    'i </w>': 2,
    'a m </w>': 1,
    'h a pp p y</w>': 1,
    'h a v e </w>': 1,
    'N </w>': 1,
    'a p p l e ss </w>': 1
}
print(refresh_bpe_vocab_by_merging_bigram(bigram, old_bpe_vocab))

{'i </w>': 2, 'a m </w>': 1, 'h a pp py</w>': 1, 'h a v e </w>': 1, 'N </w>': 1, 'a p p l e ss </w>': 1}


In [11]:
def get_bpe_tokens(bpe_vocab: Dict[str, int]) -> List[Tuple[str, int]]:
    """
    根据"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回，例如：  
    输入 
    ```python
    bpe_vocab = {
        'i</w>': 2,
        'a m </w>': 1,
        'ha pp y </w>': 1,
        'ha v e </w>': 1,
        'N </w>': 1,
        'a pp l e s </w>': 1
    }
    ```
    得到
    ```
    [
        ('i</w>', 2),
        ('ha', 2),
        ('pp', 2),
        ('a', 2),
        ('m', 1),
        ('</w>', 5),
        ('y', 1),
        ('v', 1),
        ('e', 2),
        ('N', 1),
        ('l', 1),
        ('s', 1)
    ]
    ```

    Args:
        bpe_vocab: Dict[str, int] - "单词分词状态->频数"的词典

    Return:
        List[Tuple[str, int]] - BPE分词和对应频数组成的List
    """

    bpe_tokens = []
    # TODO: 完成函数体【2分】
    tokens_dict = dict()
    for word,num in bpe_vocab.items():
        for key in word.split(' '):
            if key not in tokens_dict.keys():
                tokens_dict[key] = num
            else:
                tokens_dict[key] += num
    for key in tokens_dict.keys():
        bpe_tokens.append((key,tokens_dict[key]))
    bpe_tokens.sort(key=lambda x: len(x[0].replace('</w>', 'w')), reverse=True)

    return bpe_tokens

In [89]:
bpe_vocab = {
    'i</w>': 2,
    'a m </w>': 1,
    'ha pp y </w>': 1,
    'ha v e </w>': 1,
    'N </w>': 1,
    'a ppp l e s </w>': 1
}

print(get_bpe_tokens(bpe_vocab))

[('ppp', 1), ('i</w>', 2), ('ha', 2), ('pp', 1), ('a', 2), ('m', 1), ('</w>', 5), ('y', 1), ('v', 1), ('e', 2), ('N', 1), ('l', 1), ('s', 1)]


In [69]:
def print_bpe_tokenize(word: str, bpe_tokens: List[Tuple[str, int]]):
    """
    根据按长度降序的BPE分词列表，将所给单词进行BPE分词，最后打印结果。
    
    思想是，对于一个待BPE分词的单词，按照长度顺序从列表中寻找BPE分词进行子串匹配，  
    若成功匹配，则对该子串左右的剩余部分递归地进行下一轮匹配，直到剩余部分长度为0，  
    或者剩余部分无法匹配（该部分整体由`"<unknown>"`代替）
    
    例1：  
    输入 `word = "supermarket"`, `bpe_tokens=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
    ]`  
    最终打印 `"su per <unknown>"`

    例2：  
    输入 `word = "shanghai"`, `bpe_tokens=[
        ("hai", 1),
        ("sh", 1),
        ("an", 1),
        ("</w>", 1),
        ("g", 1)
    ]`  
    最终打印 `"sh an g hai </w>"`

    Args:
        word: str - 待分词的单词
        bpe_tokens: List[Tuple(str, int)] - BPE分词和对应频数组成的列表
    """

    # TODO: 请尝试使用递归函数定义该分词过程【2分】
    def bpe_tokenize(sub_word: str) -> str:
        for token, _ in bpe_tokens:
            if token in sub_word:
                position = sub_word.find(token)
                if len(token) == len(sub_word):
                    return token
                elif position == 0 :
                    return token + ' ' + bpe_tokenize(sub_word[len(token):])
                elif position + len(token) == len(sub_word):
                    return bpe_tokenize(sub_word[:position]) + ' ' + token
                else:
                    return bpe_tokenize(sub_word[:position]) + ' ' + token + ' ' + bpe_tokenize(sub_word[position+len(token):])
        return '<unknown>'

    res = bpe_tokenize(word + '</w>')
    res = '"'+res+'"'
    print(res)


In [70]:
word1 = "supermarket"
bpe_tokens1=[
        ("su", 20),
        ("are", 10),
        ("per", 30),
        ('</w>', 1)
    ]
word2 = "shanghai"
bpe_tokens2=[
    ("hai", 1),
    ("sh", 1),
    ("an", 1),
    ("</w>", 1),
    ("g", 1)
    ]

print_bpe_tokenize(word1, bpe_tokens1)
print_bpe_tokenize(word2, bpe_tokens2)

"su per <unknown> </w>"
"sh an g hai </w>"


开始读取数据集并训练BPE分词器：

In [42]:
with open("data/news.2007.en.shuffled.deduped.train", encoding="utf-8") as f:
    training_corpus = list(map(lambda l: l.strip(), f.readlines()[:1000]))

print("Loaded training corpus.")
print(training_corpus) # 1000 words

Loaded training corpus.
['The journey from the Clyde to Inverness was promoted as the Royal Route after Queen Victoria and Prince Albert made the trip in 1847 and the monarch also sailed on the Gondolier in 1873.', 'The warning from the Fed, which cut interest rates earlier this week, triggered concern that it might hold off further rate cuts or even consider raising them if inflation accelerates.', "When the witness' boyfriend tried to seek help from security, he was jumped by the posse of Smith's boyfriend and left with a fractured face.", 'There are concrete limits to growth and no one wants to admit that.', "KABUL, Afghanistan (CNN) -- Ten people -- including three members of Afghanistan's parliament -- were killed in a suicide bomb blast Tuesday as they visited a sugar plant in Baghlan province, another member of parliament told CNN.", 'When the Army realises someone has become a liability, they get rid of him.', 'A "lagged effect of high unemployment" -- which reached a postwar h

In [105]:
training_iter_num = 300

# 将单词每个字母以空格隔开、结尾加上</w>后，构建带频数的词典
training_bpe_vocab = build_bpe_vocab(training_corpus)
# print(training_bpe_vocab)

for i in range(training_iter_num):
    # TODO: 完成训练循环内的代码逻辑【2分】
    # 统计各种bigram的频次，选择最高的频次合并，一共进行300轮
    bigram_freq = get_bigram_freq(training_bpe_vocab)    # binary频次划分
    max_freq = 0  #记录最大频次

    max_bigram = None   #记录最大频次的bigram
    for bigram, freq in bigram_freq.items():
        if freq > max_freq:
            max_freq = freq
            max_bigram = bigram
    if max_freq == 1:
        # 如果已经到达了1就不再合并，跳出循环
        break
    training_bpe_vocab = refresh_bpe_vocab_by_merging_bigram(max_bigram, training_bpe_vocab)
# 根据给定的"单词分词状态->频数"的词典，返回所得到的BPE分词列表，并将该列表按照分词长度降序排序返回
training_bpe_tokens = get_bpe_tokens(training_bpe_vocab) # 输进去的是带分词状态的单词+词频

In [103]:
# 添加的用于测试的代码
print(training_bpe_vocab)
for token in training_bpe_tokens:
    if token[0] == 'atur':
        print(token)
        break

{'the</w>': 1218, 'jour ne y</w>': 1, 'from</w>': 99, 'cl y de</w>': 1, 'to</w>': 523, 'inver ne ss</w>': 1, 'was</w>': 121, 'promo ted</w>': 1, 'as</w>': 120, 'ro y al</w>': 7, 'rou te</w>': 1, 'after</w>': 44, 'qu e en</w>': 2, 'vic tori a</w>': 1, 'and</w>': 502, 'p rin ce</w>': 1, 'al ber t</w>': 1, 'm ade</w>': 15, 'tri p</w>': 4, 'in</w>': 424, 'N</w>': 772, 'mon ar ch</w>': 2, 'also</w>': 44, 's ailed</w>': 2, 'on</w>': 174, 'gon doli er</w>': 1, 'w ar ning</w>': 2, 'fed</w>': 5, 'which</w>': 49, 'cu t</w>': 10, 'intere st</w>': 11, 'r ates</w>': 7, 'ear li er</w>': 11, 'this</w>': 69, 'we e k</w>': 12, 'tri g gered</w>': 1, 'con cer n</w>': 3, 'that</w>': 198, 'it</w>': 113, 'mi ght</w>': 10, 'hold</w>': 3, 'o ff</w>': 16, 'fur ther</w>': 6, 'r ate</w>': 7, 'cu ts</w>': 4, 'or</w>': 46, 'e ven</w>': 10, 'con si der</w>': 3, 'r aising</w>': 1, 'them</w>': 28, 'if</w>': 30, 'in f l ation</w>': 3, 'ac celer ates</w>': 1, 'when</w>': 47, 'w it ne ss</w>': 2, "' </w>": 20, 'bo y fri

测试BPE分词器的分词效果：

In [104]:
test_word = "naturallanguageprocessing"

print("naturallanguageprocessing 的分词结果为：")
print_bpe_tokenize(test_word, training_bpe_tokens)

naturallanguageprocessing 的分词结果为：
"n atur al lan gu age pro ce s sing</w>"


### 实验总结
> TODO：请在这里填写实验总结。

实验二中，使用BPE分词方法进行分词。流程大体为，在给定的包含许多句子和单词的训练集中，对每一个单词做拆分操作，按照多个字母相连的频次高低的顺序统计形成词表，再使用该词表按照从长到短的顺序进行匹配分词。具体而言，在统计词表时，先将每一个单词拆分，作为初始的unigram，再统计两个unigram相连的频次，在每一轮中选择频次最大的两个unigram合并成为新的unigram，循环执行生成词表。

在完成代码过程中，我对于每一个函数，都将示例提取出来并进行适当调整用于检验函数功能。我保留了这些调试的代码和输出，他们很好的帮助我进行debug。

对于最终结果，分词效果不太乐观。我个人尝试了更高的迭代次数，效果依旧不好，分析原因可能是因为，在最终的测试中，我们实际上是将三个独立的单词合并在一起构成了一个新的人造单词，但是在用训练集生成词表时，由于每一个单词后面都添加了一个"</w>"，就会使得构成的词表中并没有出现过（没有添加"</w>"）的该单词，而即使去掉了“</w>”，如果该单词没有出现过，或者出现频率过少，或者迭代轮数少，都会导致无法分词成功。

总体来看，实验二的难度高于实验一，主要在于代码的实现上有一个递归函数。本次实验加深了我对BPE算法的理解，并促进了我对更多算法细节的思考，收获很多。